In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib
from IPython.display import display, display_pretty, Javascript, HTML

# Flow

- [Download and preprocess county-level results](#Townhall-data)
- [Downlaod and preprocess county-level metadata](#Census-data)
- [Combine datasets](#Combine-data)
- [Export county-level results](#Export-data)
- [Visualize](#Visualize)

## Townhall data

In [2]:
# each page has a summary table that rolls up results at the state level
# get rid of it
def cond(x):
    if x:
        return x.startswith("table ec-table") and not "table ec-table ec-table-summary" in x
    else:
        return False

In [3]:
# list of state abbreviations
states = ['AL','AK','AZ','AR','CA','CO','CT','DC','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

# headers for csv export
data = [['state_abbr', 'county_name', 'party', 'votes_total']]

In [4]:
# loop through each state's web page http://townhall.com/election/2016/president/%s/county, where %s is the state abbr
for state in states:
    r = urllib.urlopen('http://townhall.com/election/2016/president/' + state + '/county').read()
    soup = BeautifulSoup(r, "html.parser")

    # loop through each <table> tag with .ec-table class
    tables = soup.findAll('table', attrs={'class':cond})

    for table in tables:
        if table.findParent("table") is None:
            table_body = table.find('tbody')

            rows = table_body.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                # first tbody tr has four td
                if len(cols) == 4:
                    # strip text from each td
                    divs = cols[0].find_all('div')
                    county = divs[0].text.strip()
                    party = cols[1]['class'][0]
                    total_votes = int(cols[2].text.strip().replace(',','').replace('-','0'))
                # all other tbody tr have three td
                else:
                    party = cols[1]['class'][0]
                    total_votes = int(cols[1].text.strip().replace(',','').replace('-','0'))
                    
                #combine each row's results
                rowData = [state,county,party,total_votes]
                data.append(rowData)

In [5]:
townhall = pd.DataFrame(data) # throw results in dataframe
new_header = townhall.iloc[0] #grab the first row for the header
townhall = townhall[1:] #take the data less the header row
townhall.columns = new_header #set the header row as the df header
townhall['votes_total'] = townhall['votes_total'].astype('float64')
print(townhall.shape[0])
townhall.head()

14188


,state_abbr,county_name,party,votes_total
1,AL,Autauga,GOP,18110.0
2,AL,Autauga,DEM,5908.0
3,AL,Autauga,IND,538.0
4,AL,Autauga,IND,105.0
5,AL,Baldwin,GOP,72780.0


In [6]:
# view by state
townhall[(townhall['state_abbr'] == 'VA')]

,state_abbr,county_name,party,votes_total
12579,VA,Accomack,GOP,8582.0
12580,VA,Accomack,DEM,6737.0
12581,VA,Accomack,LIB,262.0
12582,VA,Accomack,IND,112.0
12583,VA,Accomack,GRN,62.0
12584,VA,Albemarle,DEM,33297.0
12585,VA,Albemarle,GOP,19232.0
12586,VA,Albemarle,LIB,2100.0
12587,VA,Albemarle,IND,1057.0
12588,VA,Albemarle,GRN,448.0


In [7]:
# view special cases
print(townhall[(townhall['state_abbr'] == 'NV') & (townhall['county_name'] == 'Carson City')])

0    state_abbr  county_name party  votes_total
7943         NV  Carson City   GOP      13125.0
7944         NV  Carson City   DEM       9610.0
7945         NV  Carson City   LIB       1159.0
7946         NV  Carson City   OTH       1122.0


In [8]:
# fix townhall county name for Washington DC, Sainte Genevieve, MO, Oglala, SD
townhall.loc[townhall['state_abbr'] =='DC', 'county_name'] = 'District of Columbia'
townhall.loc[townhall['county_name'] == 'Sainte Genevieve', 'county_name'] = 'Ste. Genevieve County'
townhall.loc[townhall['county_name'] == 'Oglala Lakota', 'county_name'] = 'Oglala'
print(townhall[(townhall['county_name'] == 'District of Columbia') | (townhall['county_name'] == 'Ste. Genevieve County') | (townhall['county_name'] == 'Oglala')])

0     state_abbr            county_name party  votes_total
1415          DC   District of Columbia   DEM     260223.0
1416          DC   District of Columbia   GOP      11553.0
1417          DC   District of Columbia   LIB       4501.0
1418          DC   District of Columbia   DCG       3995.0
7227          MO  Ste. Genevieve County   GOP       5495.0
7228          MO  Ste. Genevieve County   DEM       2540.0
7229          MO  Ste. Genevieve County   LIB        323.0
7230          MO  Ste. Genevieve County   GRN         55.0
7231          MO  Ste. Genevieve County   OTH         46.0
10780         SD                 Oglala   DEM       2504.0
10781         SD                 Oglala   GOP        241.0
10782         SD                 Oglala   LIB        104.0
10783         SD                 Oglala   OTH         47.0


In [9]:
# change 'Co.' to 'County' in county_name to match census county name
townhall['county_name'] = townhall['county_name'].apply(lambda x: x.replace('Co.','County').strip())
print(townhall[(townhall['state_abbr'] == 'NV') & (townhall['county_name'] == 'Carson City')])

0    state_abbr  county_name party  votes_total
7943         NV  Carson City   GOP      13125.0
7944         NV  Carson City   DEM       9610.0
7945         NV  Carson City   LIB       1159.0
7946         NV  Carson City   OTH       1122.0


In [10]:
# combine state and county names
townhall['combined'] = townhall['state_abbr'] + townhall['county_name'].apply(lambda x: x.replace(' ','').lower())
print(townhall[(townhall['state_abbr'] == 'NV') & (townhall['county_name'] == 'Carson City')])

0    state_abbr  county_name party  votes_total      combined
7943         NV  Carson City   GOP      13125.0  NVcarsoncity
7944         NV  Carson City   DEM       9610.0  NVcarsoncity
7945         NV  Carson City   LIB       1159.0  NVcarsoncity
7946         NV  Carson City   OTH       1122.0  NVcarsoncity


## Census data

In [11]:
# county_fips data from https://www.census.gov/geo/reference/codes/cou.html
census = pd.read_csv('http://www2.census.gov/geo/docs/reference/codes/files/national_county.txt',sep=',',header=None, dtype=str)
census.columns = ['state_abbr', 'state_fips', 'county_fips', 'county_name', 'fips_class_code']
print(census.shape)
census.head()

(3235, 5)


,state_abbr,state_fips,county_fips,county_name,fips_class_code
0,AL,01,001,Autauga County,H1
1,AL,01,003,Baldwin County,H1
2,AL,01,005,Barbour County,H1
3,AL,01,007,Bibb County,H1
4,AL,01,009,Blount County,H1


In [12]:
# view by state
census[(census['state_abbr'] == 'NV') & (census['county_name'] == 'Carson City')]

,state_abbr,state_fips,county_fips,county_name,fips_class_code
1763,NV,32,510,Carson City,C7


In [13]:
# veiw special cases
print(census[(census['state_abbr'] == 'NV') & (census['county_name'] == 'Carson City')])

     state_abbr state_fips county_fips  county_name fips_class_code
1763         NV         32         510  Carson City              C7


In [14]:
# change Shannon County, SD to Oglala County, SD
# http://rapidcityjournal.com/news/local/it-s-official-oglala-lakota-county-replaces-shannon-county-name/article_ac5c2369-3fea-5f94-9898-b007b7ddf22c.html
# townhall.loc[townhall['county_name'] == 'Sainte Genevieve', 'county_name'] = 'Ste. Genevieve County'
census.loc[(census['county_name'] == 'Shannon County') & (census['state_abbr'] == 'SD'), 'county_name'] = 'Oglala County'
census[(census['state_abbr'] == 'SD')]

,state_abbr,state_fips,county_fips,county_name,fips_class_code
2362,SD,46,003,Aurora County,H1
2363,SD,46,005,Beadle County,H1
2364,SD,46,007,Bennett County,H1
2365,SD,46,009,Bon Homme County,H1
2366,SD,46,011,Brookings County,H1
2367,SD,46,013,Brown County,H1
2368,SD,46,015,Brule County,H1
2369,SD,46,017,Buffalo County,H1
2370,SD,46,019,Butte County,H1
2371,SD,46,021,Campbell County,H1


In [15]:
# change county_name values in townhall data to match 'county_name' values for C7 fips class code cities
# get and transform C7 city names
cities = (census['state_abbr'][(census['fips_class_code'] == 'C7')] + census['county_name'][(census['fips_class_code'] == 'C7')].apply(lambda x: x.replace('city','').replace(' ','').lower()))
cities

1216          MDbaltimore
1597           MOst.louis
1763         NVcarsoncity
2915         VAalexandria
2916            VAbedford
2917            VAbristol
2918         VAbuenavista
2919    VAcharlottesville
2920         VAchesapeake
2921    VAcolonialheights
2922          VAcovington
2923           VAdanville
2924            VAemporia
2925            VAfairfax
2926        VAfallschurch
2927           VAfranklin
2928     VAfredericksburg
2929              VAgalax
2930            VAhampton
2931       VAharrisonburg
2932           VAhopewell
2933          VAlexington
2934          VAlynchburg
2935           VAmanassas
2936       VAmanassaspark
2937       VAmartinsville
2938        VAnewportnews
2939            VAnorfolk
2940             VAnorton
2941         VApetersburg
2942           VApoquoson
2943         VAportsmouth
2944            VAradford
2945           VArichmond
2946            VAroanoke
2947              VAsalem
2948           VAstaunton
2949            VAsuffolk
2950      VA

In [16]:
# loop through 'combined' column and compare to cities series to add 'city' to H1 fips class code to townhall data
for i, row in cities.iteritems():
    if row <> 'NVcarsoncity':
        townhall.loc[townhall['combined'] == row, 'combined'] = row + 'city'
    
print(townhall[(townhall['combined'] == 'NVcarsoncity')])

0    state_abbr  county_name party  votes_total      combined
7943         NV  Carson City   GOP      13125.0  NVcarsoncity
7944         NV  Carson City   DEM       9610.0  NVcarsoncity
7945         NV  Carson City   LIB       1159.0  NVcarsoncity
7946         NV  Carson City   OTH       1122.0  NVcarsoncity


In [17]:
# remove 'county' from 'combined' column of C7 fips class code counties in townhall
townhall['combined'] = townhall['combined'].str.replace('county','')
print(townhall[(townhall['county_name'] == 'Oglala')])

0     state_abbr county_name party  votes_total  combined
10780         SD      Oglala   DEM       2504.0  SDoglala
10781         SD      Oglala   GOP        241.0  SDoglala
10782         SD      Oglala   LIB        104.0  SDoglala
10783         SD      Oglala   OTH         47.0  SDoglala


In [18]:
census['combined'] = census['state_abbr'] + census['county_name'].apply(lambda x: x.replace('County','').replace('Parish','').replace(' ','').lower())
print(census[(census['state_abbr'] == 'NV') & (census['county_name'] == 'Carson City')])
# print(census[(census['state_abbr'] == 'VA') & (census['county_name'] == 'Bedford County')])

     state_abbr state_fips county_fips  county_name fips_class_code  \
1763         NV         32         510  Carson City              C7   

          combined  
1763  NVcarsoncity  


In [19]:
# return sum of votes by state and county
townhall['total_votes'] = townhall['votes_total'].groupby(townhall['combined']).transform('sum')
townhall_counties = townhall.drop('votes_total',axis=1)

# view dataset by selected state
print(townhall_counties[(townhall_counties['state_abbr'] == 'NV') & (townhall_counties['county_name'] == 'Carson City')])

0    state_abbr  county_name party      combined  total_votes
7943         NV  Carson City   GOP  NVcarsoncity      25016.0
7944         NV  Carson City   DEM  NVcarsoncity      25016.0
7945         NV  Carson City   LIB  NVcarsoncity      25016.0
7946         NV  Carson City   OTH  NVcarsoncity      25016.0


## Combine data

In [20]:
# join census and townhall data on the 'combined' column
right = townhall.set_index('combined')
left = census.set_index('combined')

combined = left.join(right, lsuffix='', rsuffix='_r')
combined = combined.reset_index()
print('Joined dataset has ' + str(combined.shape[0]) + ' items')

# view data by selected state
combined[(combined['state_abbr'] == 'NV') & (combined['fips_class_code'] == 'C7')]


Joined dataset has 14306 items


,combined,state_abbr,state_fips,county_fips,county_name,fips_class_code,state_abbr_r,county_name_r,party,votes_total,total_votes
8852,NVcarsoncity,NV,32,510,Carson City,C7,NV,Carson City,GOP,13125.0,25016.0
8853,NVcarsoncity,NV,32,510,Carson City,C7,NV,Carson City,DEM,9610.0,25016.0
8854,NVcarsoncity,NV,32,510,Carson City,C7,NV,Carson City,LIB,1159.0,25016.0
8855,NVcarsoncity,NV,32,510,Carson City,C7,NV,Carson City,OTH,1122.0,25016.0


In [21]:
# return unique dataset
county_level_combined = combined.drop_duplicates()
print('Combined dataset has ' + str(county_level_combined.shape[0]) + ' total items')
                                                   
# return only D and R results
county_level_combined = county_level_combined[(county_level_combined['party'] == 'GOP') | (county_level_combined['party'] == 'DEM')]
print('Filtered dataset has ' + str(county_level_combined.shape[0]) + ' D and R items')

# flatten dataset by adding votes by R and D columns
county_level_combined['votes_dem'] = county_level_combined['votes_total'].where(county_level_combined['party'] == 'DEM',0).astype('float32')
county_level_combined['votes_gop'] = county_level_combined['votes_total'].where(county_level_combined['party'] == 'GOP',0).astype('float32')

# drop party and party-level totals and other columns
county_level_combined.drop(['party','votes_total','state_abbr_r','county_name_r'], axis=1, inplace=True)
# total_results = county_level_combined.drop(['party','votes_total','fips_class_code','state_abbr_r','county_name_r', 'votes_dem', 'votes_gop'], axis=1, inplace=True)
county_level_combined[(county_level_combined['state_abbr'] == 'NV') & (county_level_combined['fips_class_code'] == 'C7')]

Combined dataset has 14305 total items
Filtered dataset has 6224 D and R items


,combined,state_abbr,state_fips,county_fips,county_name,fips_class_code,total_votes,votes_dem,votes_gop
8852,NVcarsoncity,NV,32,510,Carson City,C7,25016.0,0.0,13125.0
8853,NVcarsoncity,NV,32,510,Carson City,C7,25016.0,9610.0,0.0


In [22]:
# gut checks

county_level_combined[(county_level_combined['state_fips'] == '15')]

,combined,state_abbr,state_fips,county_fips,county_name,fips_class_code,total_votes,votes_dem,votes_gop
2272,HIhawaii,HI,15,001,Hawaii County,H1,64865.0,41258.0,0.0
2273,HIhawaii,HI,15,001,Hawaii County,H1,64865.0,0.0,17500.0
2277,HIhonolulu,HI,15,003,Honolulu County,H1,285683.0,175634.0,0.0
2278,HIhonolulu,HI,15,003,Honolulu County,H1,285683.0,0.0,90296.0
2283,HIkauai,HI,15,007,Kauai County,H1,26335.0,16456.0,0.0
2284,HIkauai,HI,15,007,Kauai County,H1,26335.0,0.0,7574.0
2288,HImaui,HI,15,009,Maui County,H1,51942.0,33479.0,0.0
2289,HImaui,HI,15,009,Maui County,H1,51942.0,0.0,13445.0


In [23]:
# pivot data to consolidate
party_pivot = pd.pivot_table(county_level_combined,index=["combined"],values=["votes_dem","votes_gop"],aggfunc=np.sum)
total_pivot = pd.pivot_table(county_level_combined,index=["combined"],values=["total_votes"],aggfunc=np.mean)

# join party and total pivots
combined_pivot = party_pivot.join(total_pivot, lsuffix='', rsuffix='_r')
print('Joined dataset has ' + str(combined_pivot.shape[0]) + ' items')
combined_pivot

# add percentages for each R and D of total votes
# calculate percentage of total vote per major candidates
combined_pivot['per_dem'] = combined_pivot['votes_dem'] / combined_pivot['total_votes']
combined_pivot['per_gop'] = combined_pivot['votes_gop'] / combined_pivot['total_votes']
combined_pivot['diff'] = abs(combined_pivot['votes_gop'] - combined_pivot['votes_dem']).map('{:,.0f}'.format)
combined_pivot['per_point_diff'] = abs(combined_pivot['per_dem'] - combined_pivot['per_gop']).map('{:,.2%}'.format)

combined_pivot

# join pivotted and unpivotted data
right = census.set_index('combined')
# left = combined_pivot.set_index('combined')

county_level_final = combined_pivot.join(right, lsuffix='', rsuffix='_r')
county_level_final = county_level_final.reset_index()
print('Joined dataset has ' + str(county_level_final.shape[0]) + ' items')
county_level_final

# create FIPS columns for visualizations
county_level_final['combined_fips'] = county_level_final['state_fips'].apply(lambda x: x.lstrip('0')) + county_level_final['county_fips']
county_level_final

# drop irrelevant columns
county_level_final = county_level_final.drop(['combined','county_fips','state_fips','fips_class_code'], axis=1)
print('Final dataset has ' + str(county_level_final.shape[0]) + ' items')
county_level_final

# view data by selected state
county_level_final[(county_level_final['state_abbr'] == 'SD')]

Joined dataset has 3112 items
Joined dataset has 3112 items
Final dataset has 3112 items


,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips
2332,340.0,974.0,1407.0,0.241649,0.692253,634,45.06%,SD,Aurora County,46003
2333,1912.0,4455.0,6772.0,0.282339,0.657856,"2,543",37.55%,SD,Beadle County,46005
2334,412.0,666.0,1149.0,0.358573,0.579634,254,22.11%,SD,Bennett County,46007
2335,704.0,2105.0,2974.0,0.236718,0.707801,"1,401",47.11%,SD,Bon Homme County,46009
2336,4879.0,6748.0,12680.0,0.384779,0.532177,"1,869",14.74%,SD,Brookings County,46011
2337,5452.0,9613.0,16113.0,0.338360,0.596599,"4,161",25.82%,SD,Brown County,46013
2338,571.0,1565.0,2288.0,0.249563,0.684003,994,43.44%,SD,Brule County,46015
2339,296.0,171.0,490.0,0.604082,0.348980,125,25.51%,SD,Buffalo County,46017
2340,693.0,3355.0,4346.0,0.159457,0.771974,"2,662",61.25%,SD,Butte County,46019
2341,105.0,704.0,831.0,0.126354,0.847172,599,72.08%,SD,Campbell County,46021


## Export data

In [24]:
county_level_final.to_csv('2016_US_County_Level_Presidential_Results.csv',sep=',')

## Visualize

In [25]:
%%javascript
// https://github.com/d3/d3/issues/1693#issuecomment-35556356
require.config({
    paths: {
        "d3": "http://d3js.org/d3.v4",
        "d3scalechoromatic": "http://d3js.org/d3-scale-chromatic.v1.min",
        "topojson": "http://d3js.org/topojson.v1.min",
        "d3legend":"https://cdnjs.cloudflare.com/ajax/libs/d3-legend/2.11.0/d3-legend.min"
  },
    shim: {
        "d3scalechoromatic": ["d3.global"],
        "d3-interpolate":["d3.global"]
  }
});

define("d3.global", ["d3"], function(_) {
  this.d3 = _;
});

<IPython.core.display.Javascript object>

In [26]:
display(HTML("""
<style>
  .tract {
    stroke: #777;
    stroke-width: 0.05px;
    pointer-events: all;
  }
  .tract:hover {
    stroke: orange;
    pointer-events: all;

  }
  .tract-border {
    fill: none;
    /*stroke: #777;*/
    stroke-width: 0.05px;
    pointer-events: none;
  }
  @media (max-width: 767px) {
    .label {
      transform: rotate(-45deg) translate(-10px, 25px);
    }
  }
  .tract-border-state {
    fill: none;
    stroke: #333;
    stroke-width: 0.5px;
    pointer-events: none;
  }
  .legend {
    font-family: sans-serif;
    font-size: 10pt;

  }
  .legendTitle {
    font-weight: bold;
    font-size:11pt;
  }
  .background {
    fill:#f5f5f5;
  }
  body {
    background: #f5f5f5;
    /*font-family: sans-serif;*/
  }
  .bar {
    font-family: sans-serif;
    font-size: 8pt;
  }
  #cityLegend {
    font-family: sans-serif;
    text-anchor: middle;
  }
  #details .background {
    fill: #f5f5f5;
    fill-opacity: 0.7;
  }
  svg {
    width:100%;
  }
</style>
<div class="row">
  <div class="col-md-1 col-xs-1"></div>
  <div class="col-md-10 col-xs-10">
    <h2>2016 Presidential General Election Results by County</h2>
    <h4>Click on the map to explore</h4>
  </div>
</div>
<div class="row">
  <div class="col-md-12 col-xs-12">
    <svg ></svg>
  </div>
</div>
<div class="row">
  <div class="col-md-1 col-xs-1"></div>
  <div class="col-md-10 col-xs-10">
    <p>
      By <a href="http://johnguerra.co">John Alexis Guerra Gómez</a>
    </p>

  </div> <!-- col-md-10 -->
  <div class="col-md-1">
  </div>

</div> <!-- .row -->
"""))

In [27]:
%%javascript

require(["d3", "topojson"], function(d3, topojson) {
    var svg = d3.select("svg"),
    width = $(document).width() * 10 / 12,
    height = $(document).height() - 200,
    margin = {
        top: 20,
        bottom: width > 767 ? 20 : 100,
        right: 20,
        left: 0
    },
    centered,
    fmt = d3.format(" >5.2%"),
    errorCount = 0;

svg.attr("width", width)
    .attr("height", height);

function ready(error, us, data) {
    if (error) throw error;


    var dictCities = {};
    data.forEach(function(d) {
        //Parse the percentages
        d["per_gop"] = +(d["per_gop"].slice(0, -1).replace(",", "."));
        d["per_dem"] = +(d["per_dem"].slice(0, -1).replace(",", "."));
        d.result = d["per_dem"] - d["per_gop"];
        d.combined_fips = +d.combined_fips;
        dictCities[d.combined_fips] = d;
    });

    var color = d3.scaleSequential(d3.interpolateRdBu)
        .domain([-1, 1]);

    // Add background
    svg.append("rect")
        .attr("class", "background")
        .attr("width", width)
        .attr("height", height)
        // .on("click", clicked);
        // To allow the zoom back
        // svg.on("click", clicked);
    var zoom = d3.zoom()
        .scaleExtent([1, 15])
        .on("zoom", zoomed);

    svg.style("pointer-events", "all")
        .call(zoom);
    var g = svg.append("g");

    function zoomed() {
        console.log(d3.event.transform);
        g.attr("transform", d3.event.transform);
    };

    var projection = d3.geoAlbersUsa()
        .scale(1280)
        .translate([width / 2, height / 2]);

    var path = d3.geoPath()
        .projection(projection);

    g.selectAll("path")
        .data(topojson.feature(us, us.objects.counties).features)
        .enter().append("path")
        .attr("class", "tract")
        .on("click", clicked)
        .on("mouseover", updateDetails)
        .style("fill", function(d) {
            var city = dictCities[d.id];
            if (city)
                return color(city.result);
            else {
                errorCount++;
                console.log(d.id + " Not found" + " errors = " + errorCount);
                return color(0);
            }
        })
        .attr("d", path)
        .append("title")
        .text(function(d) {
            var city = dictCities[d.id],
                county,
                state;
                
            // var msg = d.id;
            if (city) {
                county = city.county_name;
                state = city.state_abbr;
                var msg = county + ', ' + state + " Difference: " + fmt(city.result);
            }
            return msg;
        });


    // g.append("g")
    //     .attr("class", "counties")
    //   .selectAll("path")
    //     .data(topojson.feature(us, us.objects.counties).features)
    //   .enter().append("path")
    //     .attr("class", "tract-border")
    //     .attr("d", path);

    g.append("path")
        .datum(topojson.mesh(us, us.objects.states, function(a, b) {
            return a !== b;
        }))
        .attr("class", "tract-border-state")
        .attr("d", path);


    // g.append("path")
    //     .datum(topojson.mesh(mapData, mapData.objects.depts, function(a, b) { return a !== b; }))
    //     .attr("class", "tract-border-state")
    //     .attr("d", pathState);

    // The details
    var wScale = d3.scaleLinear()
        .domain([-1, 1])
        .range([-width / 3, width / 3]);
    var details_layer = svg.append("g")
        .attr("id", "details")
        .attr("transform", "translate(" + (width / 2 - 100) + ", 30)");
    details_layer.append("rect")
        .attr("class", "background")
        .attr("transform", "translate(" + (-wScale.range()[1] + 100) + ", -20)")
        .attr("width", wScale.range()[1] * 2 + 70)
        .attr("rx", 5)
        .attr("ry", 5)
        .attr("height", 60);
    details_layer.append("text")
        .attr("id", "cityLegend")
        .text("Difference")
        .attr("transform", "translate(100, 0)");

    var detailsBars = details_layer.selectAll("bar")
        .data([0.4978, -0.5021])
        .enter()
        .append("g")
        .attr("class", "bar");
    detailsBars
        .append("rect")
        .attr("width", 0)
        .attr("height", width > 767 ? 20 : 10)
        .attr("x", 100)
        .attr("y", 10)
        .style("fill", color)
        .transition()
        .duration(500)
        .attr("x", function(d) {
            return d > 0 ? 100 : 100 - wScale(-d);
        })
        .attr("width", function(d) {
            return d > 0 ? wScale(d) : wScale(-d);
        });
    detailsBars.append("text")
        .text(function(d) {
            return (d > 0 ? "" : "Rep ") +
                fmt(d > 0 ? d : -d) +
                (d > 0 ? " Dem" : "");
        })
        .attr("dx", function(d) {
            return d > 0 ? 5 : -5;
        })
        .attr("dy", 24)
        .attr("x", 100)
        .style("text-anchor", function(d) {
            return d > 0 ? "start" : "end";
        })
        .transition()
        .duration(500)
        .attr("x", function(d) {
            return d > 0 ? 100 + wScale(d) : 100 - wScale(-d);
        });



    // The legend
    svg.append("g")
        .attr("class", "legend")
        .attr("transform",
            width > 767 ?
            "translate(" + (width - margin.right - 150) + ",100)" :
            "translate(" + (width / 2 - 100) + "," + (height - 120) + ")"
        );

    var legendLinear = d3.legendColor()
        // .shapeWidth(30)
        .cells(7)
        .orient(width > 767 ? "vertical" : "horizontal")
        .title("Diferencia")
        .labels([
            " 100.00% Dem",
            "  66.67%",
            "  33.33%",
            "   0.00%",
            "  33.33%",
            "  66.67%",
            " 100.00% Rep",
        ].reverse())
        .labelFormat(fmt)
        .ascending(true)
        .labelAlign("end")
        .scale(color);

    svg.select(".legend")
        .call(legendLinear);

    // When clicked, zoom in
    function clicked(d) {
        updateDetails(d);
        var x, y, k;

        // Compute centroid of the selected path
        if (d && centered !== d) {
            // if (d) {
            var centroid = path.centroid(d);
            x = centroid[0];
            y = centroid[1];
            // k = zoom.scaleExtent()[1];
            k = 10;
            centered = d;
        }
        else {
            x = width / 2;
            y = height / 2;
            k = 1;
            centered = null;
        }



        // Manually Zoom
        svg.transition()
            .duration(750)
            .call(zoom.transform, d3.zoomIdentity
                .translate(width / 2, height / 2)
                .scale(k)
                .translate(-x, -y));
    }

    function updateDetails(d) {

        var data = [0.4978, -0.5021],
            name = "Difference " + fmt(data[0] + data[1]),
            state,
            county,
            city;
            
        if (d) {
            city = dictCities[d.id];
            if (city) {
                county = city['county_name'];
                state = city['state_abbr'];
                data = [city["per_dem"], -city["per_gop"]];
                name = county + ', ' + state + " Difference: " + fmt(data[0] + data[1]);
            }
        }
        // console.log(data);
        // console.log(name);
        var detailsBars = details_layer
            .selectAll(".bar")
            .data(data);

        detailsBars.select("rect")
            .transition()
            .duration(500)
            .attr("x", function(d) {
                return d > 0 ? 100 : 100 - wScale(-d);
            })
            .attr("width", function(d) {
                return d > 0 ? wScale(d) : wScale(-d);
            })
            .style("fill", color);

        detailsBars.select("text")
            .text(function(d) {
                return (d > 0 ? "" : "Rep ") +
                    fmt(d > 0 ? d : -d) +
                    (d > 0 ? " Dem" : "")
            })
            .transition()
            .duration(500)
            .attr("x", function(d) {
                return d > 0 ? 100 + wScale(d) : 100 - wScale(-d);
            })


        details_layer.select("#cityLegend").text(name);

    }
}

d3.queue()
    .defer(d3.json, "https://raw.githubusercontent.com/john-guerra/US_Elections_Results/master/us.json")
    .defer(d3.csv, "https://raw.githubusercontent.com/tonmcg/US_Elections_Results/master/2016_US_County_Level_Presidential_Results.csv")
    .await(ready);
    
});

<IPython.core.display.Javascript object>